# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [30]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy 
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import os 

from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\2021249\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\2021249\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
# os.chdir('..')

In [17]:
def load_data():
    engine = create_engine('sqlite:///data/DisasterResponse.db')
    df = pd.read_sql_table('Messages',con=engine)
    X = df['message']
    y = df[df.columns[4:]]
    return X, y

### 2. Write a tokenization function to process your text data

In [21]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
def model_pipeline():    
    # model_pipeline
    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
         ('clf',MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))])
    # hyper-parameter grid
    parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.75, 1.0)}
    
    # create model
    model = GridSearchCV(estimator=pipeline,
                        param_grid=parameters,
                        verbose=3,
                        cv=3)

    # cv = GridSearchCV(pipeline, param_grid=parameters)
    return model

In [34]:
model = model_pipeline()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [41]:
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = model_pipeline()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(model.score(X_test,y_test))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\2021249\3-Resources\Apps\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.278, total=  15.3s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.2s remaining:    0.0s
C:\Users\2021249\3-Resources\Apps\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.281, total=  17.5s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


...

[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.265, total=  18.4s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2) ......................


C:\Users\2021249\3-Resources\Apps\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.278, total=  20.2s


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  3.6min finished
C:\Users\2021249\3-Resources\Apps\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


0.29356118400976505


In [ ]:
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    model = model_pipeline()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(model.score(X_test,y_test))
main()

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [47]:
model.predict(pd.Series("There is a flood"))

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [54]:
def displayall(df):
    ''' Takes in a df and prints all columns and rows, max col size '''#
    with pd.option_context("display.max_rows",1000,"display.max_columns",1000,'display.max_colwidth', 1000):
        display(df)


In [58]:
displayall(pd.DataFrame(model.predict(pd.Series("We need things to eat")),columns=y_test.columns))

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [42]:
y_pred_df = pd.DataFrame(y_pred,columns=y_test.columns)
for col in y_test:
    print(col)
    print(classification_report(y_test[col],y_pred_df[col]))

related
              precision    recall  f1-score   support

           0       0.66      0.52      0.59      1592
           1       0.85      0.92      0.88      4912
           2       0.80      0.24      0.37        50

    accuracy                           0.82      6554
   macro avg       0.77      0.56      0.61      6554
weighted avg       0.81      0.82      0.81      6554

request
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      5436
           1       0.78      0.60      0.68      1118

    accuracy                           0.90      6554
   macro avg       0.85      0.78      0.81      6554
weighted avg       0.90      0.90      0.90      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\2021249\3-Resources\Apps\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6495
           1       0.80      0.07      0.12        59

    accuracy                           0.99      6554
   macro avg       0.90      0.53      0.56      6554
weighted avg       0.99      0.99      0.99      6554

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6480
           1       0.00      0.00      0.00        74

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98  

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
import pickle

In [ ]:
pickle.dumps??

In [ ]:
aparameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.